<a href="https://colab.research.google.com/github/LLMUSER/on-device-tests/blob/main/Langchain%2BSemantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken pypdf unstructured[local-inference] gradio pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.1/271.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [2]:
!pip install langchain
!pip install docx2txt
!pip install chromadb
!pip install sentence_transformers

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=f0c10d58af90a87e97a7294b31387ab2d37b60df58573a28cee8f4933fdc2374
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
    

In [3]:
from langchain.document_loaders import DirectoryLoader

In [4]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader,UnstructuredMarkdownLoader,CSVLoader,UnstructuredHTMLLoader,UnstructuredExcelLoader,TextLoader,JSONLoader,Docx2txtLoader

pdf_loader = DirectoryLoader('/content/data', glob="**/*.pdf",loader_cls=PyPDFLoader)
readme_loader = DirectoryLoader('/content/data', glob="**/*.md",loader_cls=UnstructuredMarkdownLoader)
txt_loader = DirectoryLoader('/content/data', glob="**/*.txt",loader_cls=TextLoader)
csv_loader = DirectoryLoader('/content/data', glob="**/*.csv",loader_cls=CSVLoader)
html_loader = DirectoryLoader('/content/data', glob="**/*.html",loader_cls=UnstructuredHTMLLoader)
xlsx_loader=DirectoryLoader('/content/data', glob="**/*.xlsx",loader_cls=UnstructuredExcelLoader)
json_loader = DirectoryLoader('/content/data',glob="**/*.json",loader_cls=JSONLoader)
docx_loader =  DirectoryLoader('/content/data',glob="**/*.docx",loader_cls=Docx2txtLoader)

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=99f9d69e05f80e92a97c49035c9f66491e5f7184ecdade361c069dd9824ffdf5
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [5]:
#Loading all the documents with their specialized loaders


#take all the loader
loaders = [pdf_loader, readme_loader, txt_loader,xlsx_loader,docx_loader]

#lets create document
documents = []
for loader in loaders:
    documents.extend(loader.load())

In [6]:
print (f'You have {len(documents)} document(s) in your data')
print (f'There are {len(documents[0].page_content)} characters in your document')

You have 22 document(s) in your data
There are 859 characters in your document


In [5]:
documents[1]

Document(page_content=" Article \nLlama 2: Early Adopters' Utilization of Meta's New \nOpen-Source Pretrained Model \nKonstantinos I. Roumeliotis 1,*, Nikolaos D. Tselikas 1 and Dimitrios K. Nasiopoulos 2 \n1 Department of Informatics and Telecommunications, University of Peloponnese, Akadimaikou G. K.  \nVla-chou Street, 22 131 Trip oli, Greece; ntsel@uop.gr \n2 Department of Agribusiness and Supply Chain Mana gement, School of Applied Economics and Social \nSciences, Agricultural University of At hens, 118 55 Athens, Gr eece; dimnas@aua.gr \n* Correspondence: k.roumeliotis@uop.gr; Tel.: +30-271-037-2216 \nAbstract: The rapidly evolving field of  artificial intelligence (AI) continues to witness the \nintroduction of innovative open-source pre-trained models, fostering advancements in various \napplications. One such model is Llama 2, an open-source pre-trained model released by Meta, which \nhas garnered significant attention among early adop ters. In addition to exploring the founda

In [7]:
#Splitting the text into chuncks so that we pass only that amount of tokens to our model
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=40) #chunk overlap seems to work better
documents = text_splitter.split_documents(documents)
print(len(documents))

59


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 912.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=48fc8f54c72cda7491e83fc94e96b8d1ef3d6b05802945a33a86d5416ac972a9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [11]:
embeddings = HuggingFaceEmbeddings(model_name="msmarco-distilbert-base-tas-b")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00
  Created wheel for chroma-hnswlib: filename=chroma_hnswlib-0.7.2-cp310-cp310-linux_x86_64.

In [12]:
#using chroma db as it runs on local machine for performing similarity search
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings)


In [13]:
#Here we are performing the similarity search
query = "Rohan Sidankar"
docs = vectorstore.similarity_search(query)


In [14]:
print(docs[0].page_content)
print(docs[1].page_content)


Happening now: Rohan Sidankar is inviting you to a video call
"Rohan Sidankar (via Google Meet)" <meetings-noreply@google.com>
i-omkar@hush1one.com
Rohan Sidankar <i-rohansidankar@hush1one.com> is inviting you to join a video call happening now No one else is in this call Join Call meet.google.com/qsz-ebfh-vxe Dial-in: (US) +1 541-897-2569
[]


Muybridge Derby: Bringing Animal Locomotion Photographs to Life with AI | Robert A. Gonsalves in Towards Data Science
Medium Daily Digest <noreply@medium.com>
i-omkar@hush1one.com
I Omkar Stories for I Omkar @i-omkar·Become a member Medium daily digest Today&#39;s highlights Towards Data Science Robert A. Gonsalves in Towards Data Science· 16 min read Muybridge Derby: Bringing
[]


Sunaz Sharaf invited you to a Trello Workspace
Sunaz Sharaf <invitation-do-not-reply@trello.com>
i-omkar@hush1one.com
Sunaz Sharaf invited you to their Workspace hushh Join them on Trello to collaborate, manage projects, and reach new productivity peaks. Go to Workspa

In [15]:
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off


In [16]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off
!sudo apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 1s (360 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 120831 

In [17]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, TextStreamer, pipeline
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [ ]:

DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()


In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)


In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


In [ ]:
result = qa_chain("What is the per share revenue for Meta during 2023?")
len(result["source_documents"])
print(result["source_documents"][0].page_content)
